# Assignment 2

## Imports

In [ ]:
import dagshub
import mlflow
import scripts.ML_utils as MLUtils

## Setup

In [ ]:
# Setup dagshub and MLFlow
dagshub.init(repo_owner="SamuelFredricBerg", repo_name="4dt907", mlflow=True)
utils = MLUtils("Project_Model")


# Add all configs used in the training (EXAMPLE BELOW)
config = {
    "n_estimators": 200,
    "max_depth": 7,
    "learning_rate": 0.05,
    "feature_set": "v2_processed",
    "data_split_seed": 42
}

## Main Code

In [ ]:
with mlflow.start_run(run_name="ADD_RUN_NAME_HERE") as run:
    mlflow.log_params(config)


    # TRAINING CODE GOES HERE
    # model = train_model()

    # Logging results
    # Create a dictionary for all results (EXAMPLE BELOW)
    results = {
        "accuracy": 0.942,
        "f1_score": 0.915,
        "precision": 0.920,
        "recall": 0.910
    }
    mlflow.log_metrics(results)

    # Logging visuals
    # Example how to below
    # plt.savefig("feature_importance.png")
    # mlflow.log_artifact("feature_importance.png")

    if utils.auto_check_challenger(run.info.run_id, metric_name="accuracy"):
        mlflow.sklearn.log_model(model, "model", registered_model_name="Project_Model")

        latest_v = utils.client.get_latest_versions("Project_Model")[0].version
        utils.client.set_registered_model_alias("Project_Model", "dev", latest_v)
        print("New model beat current @dev uploading to DagsHub")
    else:
        print("Did not beat current @dev, model not uploaded to Dagshub")